### Importação de Bibliotecas

In [96]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pickle

### Leitura e Tratamento dos Dados

In [2]:
jogosCopa   = pd.read_csv('archive/WorldCupMatches.csv',encoding='utf-8')
ranking     = pd.read_csv('archive/fifa_ranking-2022-10-06.csv.zip')
jogos       = pd.read_csv('archive/results.csv')
jogos1      = pd.read_csv('archive/results.csv')

In [3]:
jogos =  pd.read_csv('archive/results.csv') #games between national teams
jogos["date"] = pd.to_datetime(jogos["date"])
jogos = jogos[(jogos["date"] < "2018-8-1")].reset_index(drop=True)
df_wc = jogos #pre-wc outcomes

rank = pd.read_csv('archive/fifa_ranking-2022-10-06.csv.zip') #rankings
rank["rank_date"] = pd.to_datetime(rank["rank_date"]) 
rank = rank[(rank["rank_date"] < "2018-1-1")].reset_index(drop=True) #selecting games from the 2022 wc cycle
rank["country_full"] = rank["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States") #ajustando nomes de algumas seleções
rank = rank.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()
rank_wc = rank #dataframe with rankings

#Making the merge
df_wc_ranked = df_wc.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked = df_wc_ranked.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked.dropna(inplace=True)
jogos = df_wc_ranked

In [4]:
import pandas as pd
import re
df =  pd.read_csv('archive/results.csv') #games between national teams
df["date"] = pd.to_datetime(df["date"])
df = df[(df["date"] >= "2018-8-1")].reset_index(drop=True) #games at the 2022 wc cycle
df_wc = df #pre-wc outcomes

rank = pd.read_csv('archive/fifa_ranking-2022-10-06.csv.zip') #rankings
rank["rank_date"] = pd.to_datetime(rank["rank_date"]) 
rank = rank[(rank["rank_date"] >= "2018-1-1")].reset_index(drop=True) #selecting games from the 2022 wc cycle
rank["country_full"] = rank["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States") #ajustando nomes de algumas seleções
rank = rank.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()
rank_wc = rank #dataframe with rankings

#Making the merge
df_wc_ranked = df_wc.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked = df_wc_ranked.merge(rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked.dropna(inplace=True)

In [5]:
def ultimosJogos(jogos,data,time,qtdJogos=5):
    return jogos[(jogos['date']<data) & ((jogos['home_team']==time) | (jogos['away_team']==time))].sort_values(by='date',ascending=False).head(qtdJogos)

def qtdVit(linha,time,rank,qtdJogos=5):

    jogosC = ultimosJogos(jogos,linha['date'],linha[time],qtdJogos)
    jogosC['Target'] =  np.where(jogosC['home_score']>jogosC['away_score'],jogosC['home_team'],
                       np.where(jogosC['home_score']<jogosC['away_score'],jogosC['away_team'],'Empate'))
    return jogosC[jogosC['Target']==linha[time]]['Target'].count()

def saldoGols(linha,time,rank):
    jogosC = ultimosJogos(jogos,linha['date'],linha[time],qtdJogos=5)
    golsTeam = jogosC.groupby(['home_team'])['home_score'].sum().reset_index()
    golsTeam.columns = ['pais','gols'] 
    golsAway = jogosC.groupby(['away_team'])['away_score'].sum().reset_index() 
    golsAway.columns = ['pais','gols'] 
    golsMarcados = pd.concat([golsTeam,golsAway])
    golsMarcados = golsMarcados[golsMarcados['pais']==linha[time]].groupby(['pais'])['gols'].sum().reset_index()

    golsSofridosTeam = jogosC.groupby(['home_team'])['away_score'].sum().reset_index()
    golsSofridosTeam.columns = ['pais','gols'] 
    golsSofridosAway = jogosC.groupby(['away_team'])['home_score'].sum().reset_index() 
    golsSofridosAway.columns = ['pais','gols'] 
    golsSofridos = pd.concat([golsSofridosTeam,golsSofridosAway])
    golsSofridos = golsSofridos[golsSofridos['pais']==linha[time]].groupby(['pais'])['gols'].sum().reset_index()

    golsMarcados['saldoDeGols'] = golsMarcados['gols']-golsSofridos['gols']
    golsMarcados = golsMarcados.drop('gols',axis=1)
    
    return golsMarcados[golsMarcados['pais']==linha[time]]['saldoDeGols']

def qtdEmp(linha,time,rank):

    jogosC = ultimosJogos(jogos,linha['date'],linha[time],qtdJogos=5)
    jogosC['Target'] =  np.where(jogosC['home_score']>jogosC['away_score'],jogosC['home_team'],
                       np.where(jogosC['home_score']<jogosC['away_score'],jogosC['away_team'],'Empate'))
    return jogosC[jogosC['Target']=='Empate']['Target'].count()

def medScore(linha,time,rank):
    jogosC = ultimosJogos(jogos,linha['date'],linha[time],qtdJogos=5)
    return (jogosC[jogosC['home_team']==linha[time]]['rank_home'].sum()+jogosC[jogosC['away_team']==linha[time]]['rank_away'].sum())

In [6]:
#Target -> 1 Vitoria da Casa,2 Vitoria  do visitante e 0  empate
df_wc_ranked['Target'] = np.where(df_wc_ranked['home_score']>df_wc_ranked['away_score'],1,0)
                        #  np.where(df_wc_ranked['home_score']<df_wc_ranked['away_score'],2,0))

In [7]:
df_wc_ranked['Target_V'] = np.where(df_wc_ranked['home_score']>df_wc_ranked['away_score'],1,
                         np.where(df_wc_ranked['home_score']<df_wc_ranked['away_score'],2,0))

In [7]:
df_wc_ranked['flag_amistoso'] = np.where(df_wc_ranked['tournament']=='Friendly',1,0)
df_wc_ranked['flag_is_campo_neutro'] = np.where(df_wc_ranked['neutral']==True,1,0)

In [8]:
golsTeam = jogos.groupby(['home_team'])['home_score'].sum().reset_index()
golsTeam.columns = ['pais','gols'] 
golsAway = jogos.groupby(['away_team'])['away_score'].sum().reset_index() 
golsAway.columns = ['pais','gols'] 
golsMarcados = pd.concat([golsTeam,golsAway])
golsMarcados = golsMarcados.groupby(['pais'])['gols'].sum().reset_index()

golsSofridosTeam = jogos.groupby(['home_team'])['away_score'].sum().reset_index()
golsSofridosTeam.columns = ['pais','gols'] 
golsSofridosAway = jogos.groupby(['away_team'])['home_score'].sum().reset_index() 
golsSofridosAway.columns = ['pais','gols'] 
golsSofridos = pd.concat([golsSofridosTeam,golsSofridosAway])
golsSofridos = golsSofridos.groupby(['pais'])['gols'].sum().reset_index()

golsMarcados['saldoDeGols'] = golsMarcados['gols']-golsSofridos['gols']
golsMarcados = golsMarcados.drop('gols',axis=1)
golsMarcados

,pais,saldoDeGols
0,Afghanistan,-49.0
1,Albania,-62.0
2,Algeria,75.0
3,American Samoa,-171.0
4,Andorra,-338.0
...,...,...
192,Vietnam,27.0
193,Wales,-27.0
194,Yemen,-123.0
195,Zambia,130.0


In [9]:
df_wc_ranked['date'] = df_wc_ranked['date'].astype(str)

In [10]:
jogos

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
0,1993-01-01,Ghana,Mali,1.0,1.0,Friendly,Libreville,Gabon,True,34.0,0.0,39.0,0.0,22.0,0.0,69.0,0.0
1,1993-01-02,Gabon,Burkina Faso,1.0,1.0,Friendly,Libreville,Gabon,False,27.0,0.0,55.0,0.0,11.0,0.0,97.0,0.0
2,1993-01-03,Burkina Faso,Mali,1.0,0.0,Friendly,Libreville,Gabon,True,11.0,0.0,97.0,0.0,22.0,0.0,69.0,0.0
3,1993-01-03,Gabon,Ghana,2.0,3.0,Friendly,Libreville,Gabon,False,27.0,0.0,55.0,0.0,34.0,0.0,39.0,0.0
4,1993-01-08,Uganda,Tanzania,3.0,1.0,Friendly,Kampala,Uganda,False,12.0,0.0,92.0,0.0,15.0,0.0,80.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18808,2017-12-15,Fiji,New Caledonia,2.0,3.0,Pacific Mini Games,Port Vila,Vanuatu,True,93.0,95.0,178.0,3.0,139.0,177.0,156.0,6.0
18809,2017-12-15,Vanuatu,Solomon Islands,3.0,2.0,Pacific Mini Games,Port Vila,Vanuatu,False,55.0,51.0,185.0,-3.0,179.0,173.0,148.0,-4.0
18810,2017-12-16,Japan,South Korea,1.0,4.0,EAFF Championship,Chōfu,Japan,False,623.0,711.0,55.0,11.0,563.0,588.0,59.0,-3.0
18811,2017-12-17,Burundi,Uganda,1.0,2.0,CECAFA Cup,Machakos,Kenya,True,216.0,233.0,138.0,9.0,471.0,503.0,74.0,4.0


In [11]:
df_wc_ranked['QtdVit5UltJHome'] = df_wc_ranked[['date','home_team','rank_home']].apply(qtdVit,args=['home_team','rank_home'],axis=1)
df_wc_ranked['QtdVit5UltJAway'] = df_wc_ranked[['date','away_team','rank_away']].apply(qtdVit,args=['away_team','rank_away'],axis=1)

df_wc_ranked['QtdVit10UltJHome'] = df_wc_ranked[['date','home_team','rank_home']].apply(qtdVit,args=['home_team','rank_home',10],axis=1)
df_wc_ranked['QtdVit10UltJAway'] = df_wc_ranked[['date','away_team','rank_away']].apply(qtdVit,args=['away_team','rank_away',10],axis=1)

In [12]:
df_wc_ranked['QtdVit2UltJHome'] = df_wc_ranked[['date','home_team','rank_home']].apply(qtdVit,args=['home_team','rank_home',2],axis=1)
df_wc_ranked['QtdVit2UltJAway'] = df_wc_ranked[['date','away_team','rank_away']].apply(qtdVit,args=['away_team','rank_away',2],axis=1)

In [13]:
df_wc_ranked['QtdEmp5UltJHome'] = df_wc_ranked[['date','home_team','rank_home']].apply(qtdEmp,args=['home_team','rank_home'],axis=1)
df_wc_ranked['QtdEmp5UltJAway'] = df_wc_ranked[['date','away_team','rank_away']].apply(qtdEmp,args=['away_team','rank_away'],axis=1)

In [14]:
df_wc_ranked['Med5UltJRankHome'] = df_wc_ranked.apply(medScore,args=['home_team','rank_home'],axis=1)
df_wc_ranked['Med5UltJRankAway'] = df_wc_ranked.apply(medScore,args=['away_team','rank_away'],axis=1)

In [15]:
df_wc_ranked['saldoGols5UltJHome'] = df_wc_ranked.apply(saldoGols,args=['home_team','rank_home'],axis=1)
df_wc_ranked['saldoGols5UltJAway'] = df_wc_ranked.apply(saldoGols,args=['away_team','rank_home'],axis=1)

In [16]:
jogosCopa = jogosCopa.drop_duplicates('MatchID')
jogosCopa = jogosCopa.dropna()
jogosCopa = jogosCopa.loc[:,['Year', 'Stage', 'Home Team Name',
  'Home Team Goals', 'Away Team Goals','Win conditions','Away Team Name',
  'Attendance', 'Half-time Home Goals','Half-time Away Goals',
  'RoundID', 'MatchID']]
jogosCopa = jogosCopa.replace('Germany FR','Germany')

In [17]:
jogosCopa['TargetCopa'] = np.where(jogosCopa['Home Team Goals']>jogosCopa['Away Team Goals'],jogosCopa['Home Team Name'],
                      np.where(jogosCopa['Home Team Goals']<jogosCopa['Away Team Goals'],jogosCopa['Away Team Name'],
                      np.where(jogosCopa.apply(lambda x: x['Home Team Name'] in x['Win conditions'],axis=1),jogosCopa['Home Team Name'],
                      np.where(jogosCopa.apply(lambda x: x['Away Team Name'] in x['Win conditions'],axis=1),jogosCopa['Away Team Name'],
                      'Empate'
                    ))))

In [18]:
df_wc_ranked = df_wc_ranked.merge(golsMarcados,how='left',right_on=['pais'],left_on=['home_team'])
df_wc_ranked = df_wc_ranked.merge(golsMarcados,how='left',right_on=['pais'],left_on=['away_team'],suffixes=("_home", "_away")).drop(['pais_home','pais_away'],axis=1).fillna(0)

In [19]:
df_wc_ranked['diff_rank'] = df_wc_ranked['rank_home']-df_wc_ranked['rank_away']
df_wc_ranked['diff_points'] = df_wc_ranked['total_points_home']-df_wc_ranked['total_points_away']

In [20]:
df_wc_ranked.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'total_points_home',
       'previous_points_home', 'rank_home', 'rank_change_home',
       'total_points_away', 'previous_points_away', 'rank_away',
       'rank_change_away', 'Target', 'flag_amistoso', 'flag_is_campo_neutro',
       'QtdVit5UltJHome', 'QtdVit5UltJAway', 'QtdVit10UltJHome',
       'QtdVit10UltJAway', 'QtdVit2UltJHome', 'QtdVit2UltJAway',
       'QtdEmp5UltJHome', 'QtdEmp5UltJAway', 'Med5UltJRankHome',
       'Med5UltJRankAway', 'saldoGols5UltJHome', 'saldoGols5UltJAway',
       'saldoDeGols_home', 'saldoDeGols_away', 'diff_rank', 'diff_points'],
      dtype='object')

In [23]:
#X = df_wc_ranked.loc[:,'total_points_home':].drop('Target',axis=1)
X = df_wc_ranked[['Target','diff_rank','diff_points',
                'saldoGols5UltJHome', 'saldoGols5UltJAway','saldoDeGols_home',
                'saldoDeGols_away']].drop('Target',axis=1)
Y =  df_wc_ranked[['Target']]
X

,diff_rank,diff_points,saldoGols5UltJHome,saldoGols5UltJAway,saldoDeGols_home,saldoDeGols_away
0,1.0,-7.00,-9.0,-2.0,-76.0,-10.0
1,17.0,-85.00,22.0,3.0,-18.0,102.0
2,-35.0,245.00,-8.0,-8.0,-43.0,-16.0
3,53.0,-192.00,-11.0,5.0,-338.0,84.0
4,114.0,-420.00,-7.0,-4.0,-59.0,-7.0
...,...,...,...,...,...,...
3343,3.0,-17.80,2.0,6.0,-62.0,-35.0
3344,11.0,-60.96,4.0,3.0,124.0,42.0
3345,-45.0,190.96,11.0,4.0,198.0,7.0
3346,-2.0,3.38,-5.0,-8.0,-20.0,-151.0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=2)

In [144]:
X_train

,diff_rank,diff_points,saldoGols5UltJHome,saldoGols5UltJAway,saldoDeGols_home,saldoDeGols_away
2100,-4.0,117.26,7.0,8.0,110.0,331.0
816,23.0,-84.00,-5.0,-3.0,-171.0,42.0
1470,-19.0,68.00,-2.0,1.0,-71.0,-43.0
743,-39.0,-82.00,-1.0,0.0,79.0,0.0
597,-62.0,246.00,2.0,1.0,126.0,-31.0
...,...,...,...,...,...,...
2347,-5.0,59.86,2.0,-7.0,126.0,74.0
1608,-16.0,65.00,2.0,-2.0,22.0,-76.0
2541,-14.0,201.60,5.0,2.0,560.0,126.0
2575,-159.0,597.36,-5.0,0.0,99.0,0.0


In [25]:
rf = RandomForestClassifier(random_state=2,max_depth=5,n_estimators=100) 
model_rf = rf.fit(X_train.fillna(-1000),y_train.fillna(0))

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [97]:
with open('pikle.pkl', 'wb') as file:
    pickle.dump(model_rf, file)

In [26]:
predict = model_rf.predict_proba(X_test.fillna(-1000))
previsao = model_rf.predict(X_test.fillna(-1000))
predict_train = model_rf.predict(X_train.fillna(-1000))
previsaoTrain = model_rf.predict_proba(X_train.fillna(-1000))

print(accuracy_score(predict_train,y_train)*100)
print(accuracy_score(previsao,y_test)*100)

73.58087921468203
68.35820895522387


In [85]:
features = X.columns

In [28]:
features_importance = zip(model_rf.feature_importances_, features)
for importance, feature in sorted(features_importance, reverse=True):
    print("%s: %f%%" % (feature, importance*100))

diff_points: 41.042114%
diff_rank: 28.410492%
saldoDeGols_home: 12.090231%
saldoDeGols_away: 10.755222%
saldoGols5UltJAway: 4.997846%
saldoGols5UltJHome: 2.704095%


In [76]:
# 
paises = ['Ghana','Uruguay','South Korea','Portugal','Serbia','Switzerland','Cameroon','Brazil','Netherlands','United States','Argentina','Australia','France','Poland','England','Senegal']

jogosCopa2022 = pd.DataFrame({  'home_team':[],
                                'away_team':[],
                                'rank_home':[],
                                'rank_away':[],
                                'total_points_home':[],
                                'total_points_away':[]
                            })
cont = 0
for i in range(0,(len(paises)),2):
    
    jogosCopa2022.loc[cont] = [paises[i],paises[i+1],
     rank[(rank['rank_date']=='2022-10-06') & (rank['country_full']==paises[i])]['rank'].values[0],
     rank[(rank['rank_date']=='2022-10-06') & (rank['country_full']==paises[i+1])]['rank'].values[0],
     rank[(rank['rank_date']=='2022-10-06') & (rank['country_full']==paises[i])]['total_points'].values[0],
     rank[(rank['rank_date']=='2022-10-06') & (rank['country_full']==paises[i+1])]['total_points'].values[0]
    ]
    cont += 1

In [78]:
golsTeam = jogos1.groupby(['home_team'])['home_score'].sum().reset_index()
golsTeam.columns = ['pais','gols'] 
golsAway = jogos1.groupby(['away_team'])['away_score'].sum().reset_index() 
golsAway.columns = ['pais','gols'] 
golsMarcados = pd.concat([golsTeam,golsAway])
golsMarcados = golsMarcados.groupby(['pais'])['gols'].sum().reset_index()

golsSofridosTeam = jogos1.groupby(['home_team'])['away_score'].sum().reset_index()
golsSofridosTeam.columns = ['pais','gols'] 
golsSofridosAway = jogos1.groupby(['away_team'])['home_score'].sum().reset_index() 
golsSofridosAway.columns = ['pais','gols'] 
golsSofridos = pd.concat([golsSofridosTeam,golsSofridosAway])
golsSofridos = golsSofridos.groupby(['pais'])['gols'].sum().reset_index()

golsMarcados['saldoDeGols'] = golsMarcados['gols']-golsSofridos['gols']
golsMarcados = golsMarcados.drop('gols',axis=1)
golsMarcados

,pais,saldoDeGols
0,Abkhazia,24.0
1,Afghanistan,-112.0
2,Albania,-219.0
3,Alderney,-60.0
4,Algeria,237.0
...,...,...
311,Yugoslavia,200.0
312,Zambia,396.0
313,Zanzibar,-247.0
314,Zimbabwe,93.0


In [83]:
jogosCopa2022['diff_rank']   = jogosCopa2022['rank_home'] - jogosCopa2022['rank_away']
jogosCopa2022['diff_points'] = jogosCopa2022['total_points_home']-jogosCopa2022['total_points_away']

In [79]:
jogosCopa2022 = jogosCopa2022.merge(golsMarcados,how='left',right_on=['pais'],left_on=['home_team'])
jogosCopa2022 = jogosCopa2022.merge(golsMarcados,how='left',right_on=['pais'],left_on=['away_team'],suffixes=("_home", "_away")).drop(['pais_home','pais_away'],axis=1).fillna(0)

In [82]:
jogosCopa2022['saldoGols5UltJHome'] = [-2,-1,-4,-9,5,3,6,1]
jogosCopa2022['saldoGols5UltJAway'] = [0,3,0,6,-1,-3,-2,0]
jogosCopa2022

,home_team,away_team,rank_home,rank_away,total_points_home,total_points_away,saldoDeGols_home,saldoDeGols_away,saldoGols5UltJHome,saldoGols5UltJAway
0,Ghana,Uruguay,61.0,14.0,1393.00,1638.71,361.0,301.0,-2,0
1,South Korea,Portugal,28.0,9.0,1530.30,1676.56,767.0,368.0,-1,3
2,Serbia,Switzerland,21.0,15.0,1563.62,1635.92,152.0,-158.0,-4,0
3,Cameroon,Brazil,43.0,1.0,1471.44,1841.30,193.0,1318.0,-9,6
4,Netherlands,United States,8.0,16.0,1694.51,1627.48,694.0,134.0,5,-1
5,Argentina,Australia,3.0,38.0,1773.88,1488.72,833.0,498.0,3,-3
6,France,Poland,4.0,26.0,1759.78,1548.59,432.0,292.0,6,-2
7,England,Senegal,5.0,18.0,1728.47,1584.38,1268.0,212.0,1,0


In [127]:
predicoes = jogosCopa2022[['diff_rank', 'diff_points', 'saldoGols5UltJHome', 'saldoGols5UltJAway',
        'saldoDeGols_home', 'saldoDeGols_away']].values

In [139]:
cont = 0
for pais_home,pais_away in zip(list(jogosCopa2022['home_team']),list(jogosCopa2022['away_team'])):
    print(f'Casa->{pais_home}:{model_rf.predict_proba([predicoes[cont]])[:,1]}     Fora-> {pais_away}:{model_rf.predict_proba([predicoes[cont]])[:,0]}')
    cont += 1

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassif

Casa->Ghana:[0.27458914]     Fora-> Uruguay:[0.72541086]
Casa->South Korea:[0.28770629]     Fora-> Portugal:[0.71229371]
Casa->Serbia:[0.44699962]     Fora-> Switzerland:[0.55300038]
Casa->Cameroon:[0.14120698]     Fora-> Brazil:[0.85879302]
Casa->Netherlands:[0.47105063]     Fora-> United States:[0.52894937]
Casa->Argentina:[0.75519356]     Fora-> Australia:[0.24480644]
Casa->France:[0.64702528]     Fora-> Poland:[0.35297472]
Casa->England:[0.59957996]     Fora-> Senegal:[0.40042004]


X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
